# Natural Language Processing with Disaster Tweets

This is the link of the competition: https://www.kaggle.com/competitions/nlp-getting-started/overview

In [5]:
# Importing libraries
import pandas as pd
import string
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

# scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

In [6]:
# This is the sample format way to submit the contest
file_loc = 'data/sample_submission.csv'
df = pd.read_csv(file_loc)
df.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


# Machine Learning

In [8]:
file_loc = 'data/train.csv'
df = pd.read_csv(file_loc)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [156]:
df[df['keyword'].isna() == False]

,id,keyword,location,text,target,clean_text
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,bbcmtd wholesale market ablaze httptcolhyxeohyc
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,always try bring heavy metal rt httptcoyaoexngw
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,africanbaze break newsnigeria flag set ablaze ...
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,cry more set ablaze
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,plus side look sky last night ablaze httptcoqq...
...,...,...,...,...,...,...
7578,10830,wrecked,NaN,@jt_ruff23 @cameronhacker and I wrecked you both,0,jtruff cameronhacker wreck
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0,three days work theyve pretty much wreck hahah...
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0,fx forex trade cramer igers word wreck disney...
7581,10833,wrecked,Lincoln,@engineshed Great atmosphere at the British Li...,0,engineshed great atmosphere british lion gig t...


In [111]:
stop_w = set(stopwords.words('english'))

def preprocessing(sentence):
    
    wl = WordNetLemmatizer()
    sentence = sentence.strip()
    sentence = sentence.lower()
    list_word = sentence.split()
    list_word = [wl.lemmatize(''.join(
                    [
                        c for c in char if not c.isdigit() 
                          and c not in string.punctuation

                ]               
            ), pos='v' )
        for char in list_word
        if char not in stop_w
        ]
    
    return ' '.join(list_word)

In [112]:
df['clean_text'] = [preprocessing(s) for s in df['text']]

In [113]:
count = 0
for sentence in df['clean_text']:
    if 'fire' in sentence:
        count += 1
print(count)

474


In [114]:
df['clean_text'].unique()

array(['deeds reason earthquake may allah forgive us',
       'forest fire near la ronge sask canada',
       'residents ask shelter place notify officer evacuation shelter place order expect',
       ...,
       'suicide bomber kill  saudi security site mosque  reuters via world  google news  wall  httptconficuloje',
       'two giant crane hold bridge collapse nearby home httptcostfmbbzfb',
       'latest home raze northern california wildfire  abc news httptcoymyrskqd'],
      dtype=object)

In [153]:
# The vectorizer model
vectorizer = TfidfVectorizer()

# Transform the text data to feature vectors
X = vectorizer.fit_transform(df['clean_text'])
y = df['target']

In [154]:


# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize and train the classifier
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

SVC(kernel='linear')

In [155]:
# Predict on the test set
y_pred = clf.predict(X_test)

# Evaluate the performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy:.4f}')
print('Classification Report:')
print(report)

Accuracy: 0.8025
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.90      0.84      1318
           1       0.83      0.68      0.74       966

    accuracy                           0.80      2284
   macro avg       0.81      0.79      0.79      2284
weighted avg       0.81      0.80      0.80      2284



In [ ]:
"""
Logistic Regression model: improved 0
list of keywords in TfidVectorizer(): improved -

"""